In [48]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import altair as alt 
import os
import gc


In [49]:

# 019
path = os.path.join("simulation_results", "selenium recommended texts", "019", "tracker_output.csv")
df = pd.read_csv(path)

df['difficult_texts'] = 10
# 020
path = os.path.join("simulation_results", "selenium recommended texts", "020", "tracker_output.csv")
df020 = pd.read_csv(path)

df020['difficult_texts'] = 20
# 020
path = os.path.join("simulation_results", "selenium recommended texts", "021", "tracker_output.csv")
df021 = pd.read_csv(path)

df021['difficult_texts'] = 0

df = df.append(df020)
df = df.append(df021)

del df020, df021
gc.collect()

df = df.loc[df['labels'] < 3000]
dfq = df.loc[df['labels'] < 1150]

df

,Unnamed: 0,labels,overall_quality_score,accuracy,elapsed_time,fully_human_labeled,difficult_texts
0,0,1,0.0,0.116953,0 days 00:00:13.837214,True,10
1,1,2,0.0,0.177309,0 days 00:00:19.632586,True,10
2,2,3,0.0,0.661609,0 days 00:00:25.122789,True,10
3,3,4,0.0,0.650923,0 days 00:00:30.701531,True,10
4,4,5,45.6%,0.661873,0 days 00:00:40.077958,True,10
...,...,...,...,...,...,...,...
2995,2995,2960,93.2%,0.921966,0 days 07:42:06.090215,True,0
2996,2996,2961,93.2%,0.921966,0 days 07:42:15.400773,True,0
2997,2997,2962,93.2%,0.922032,0 days 07:42:24.517483,True,0
2998,2998,2963,93.2%,0.922032,0 days 07:42:33.776693,True,0


In [50]:
def generate_speed_accuracy_chrt(df_, lower_bound, upper_bound, accuracy_lower_bound=0.):
    chrt = alt.Chart(df_, title="Improving Speed vs Accuracy using Difficult Texts Function").mark_line().encode(
        x=alt.X("labels", scale=alt.Scale(domain=[lower_bound, upper_bound])),
        y=alt.Y("accuracy", scale=alt.Scale(domain=[accuracy_lower_bound, 1.])),
        color=alt.Color("difficult_texts:N", title="Number of Difficult Texts"),
        tooltip=['accuracy', 'difficult_texts', 'labels']
    ).properties(
        height=200,
        width=400,
        background="silver"
    ).configure_axis(
        grid=False,
        labelFontSize=14,
        titleFontSize=16
    ).configure_view(
        strokeOpacity=0
    ).configure_title(
        fontSize=20
    )
    return chrt

In [51]:
chrt = generate_speed_accuracy_chrt(dfq, 0, 1150)
chrt

alt.Chart(...)

In [54]:
dfq = dfq.loc[dfq['labels'] > 200]
chrta = generate_speed_accuracy_chrt(dfq, 200, 1150, accuracy_lower_bound=0.8)
chrta

alt.Chart(...)

In [73]:
accuracy_threshold = 0.92
difficult_texts = list(df['difficult_texts'].unique())#.sort()
difficult_texts.sort()

df_threshold = pd.DataFrame(columns=['Difficult_Texts', 'Labels Required'])

for dt in difficult_texts:
    res = {}
    res['Difficult_Texts'] = dt
    res['Labels Required'] = min(df.loc[(df['difficult_texts']==dt) & (df['accuracy']>=accuracy_threshold), "labels"])
    df_threshold = df_threshold.append(res, ignore_index=True)

df_threshold

,Difficult_Texts,Labels Required
0,0,1737
1,10,889
2,20,782


In [93]:
alt.Chart(df_threshold, title="Individual Labels Required to Get to Threshold Accuracy on Test Set").mark_bar().encode(
    x=alt.X('Difficult_Texts:N', title="Difficult Texts"),
    y="Labels Required:Q",
    # The highlight will be set on the result of a conditional statement
    color=alt.condition(
        alt.datum.Difficult_Texts == 0,  
        alt.value('red'),     
        alt.value('green')   
    )
).properties(
        height=200,
        width=300,
        background="silver"
    ).configure_axis(
        grid=False,
        labelFontSize=14,
        titleFontSize=15, 
        labelAngle=0
    ).configure_view(
        strokeOpacity=0
    ).configure_title(
        fontSize=17
    )

alt.Chart(...)